In [60]:
%matplotlib inline

import os
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from scipy.signal import resample, butter, lfilter
from scipy.io.wavfile import read as wav_read
from scipy.fftpack import fft

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import LabelKFold
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
people = []
signals = []
labels = []

pwd = 'ibs_true/'
files = os.listdir(pwd)
for f in files:
    w = wav_read(pwd + f)[1]
    if len(w) < 20000:
        # filter some short signals
        continue
    people.append(int(f.split('_')[0]))
    signals.append(w)
    labels.append(1)
    
pwd = 'ibs_false/'
files = os.listdir(pwd)
for f in files:
    w = wav_read(pwd + f)[1]
    if len(w) < 20000:
        # filter some short signals
        continue
    people.append(int(f.split('_')[0]))
    signals.append(w)
    labels.append(0)
    
people = np.array(people)
signals = np.array(signals)
labels = np.array(labels)

In [3]:
# people = []
# signals = []
# labels = []

# f = open('new_data/list.txt', 'r')
# l = f.readline()
# l = f.readline().strip().split('\t')

# while l:
#     if not l[0]:
#         break
#     people_id = int(l[0])
#     label = int(l[1])
#     people_signals = l[5:]
#     for sig in people_signals:
#         if label:
#             pwd = 'new_data/True/filtered/' + sig + '-1K.wav'
#         else:
#             pwd = 'new_data/False/filtered/' + sig + '-1K.wav'
#         w = wav_read(pwd)[1]
#         if len(w) < 20000:
#             # filter some short signals
#             continue
#         signals.append(w)
#         people.append(people_id)
#         labels.append(label)
#     l = f.readline().strip().split('\t')
    
# people = np.array(people)
# signals = np.array(signals)
# labels = np.array(labels)

In [4]:
cv = LabelKFold(people, n_folds=20)

Разделим выборку (сид подобран так, чтобы баланс в целевой переменной сохранялся).

In [6]:
np.random.seed(10)
# np.random.seed(55)
index = np.arange(0, 20)
index = np.random.permutation(index)
train_mask = np.array([i in index[:16] for i in cv.idxs])
test_mask = ~train_mask

In [7]:
train_data = signals[train_mask]
train_labels = labels[train_mask]
test_data = signals[test_mask]
test_labels = labels[test_mask]

In [8]:
np.mean(train_labels)

0.30023640661938533

In [9]:
np.mean(test_labels)

0.3392568659127625

In [111]:
slice_len = 1000

Подготовим выборку для тестирования обученной сети.

In [175]:
def preprocess_signal(signal):
    signal = np.array(signal, dtype=np.float32)
    signal -= np.mean(signal)
    signal = signal / (np.max(signal) + 1e-10)
    return signal

In [176]:
random.seed(123)

testX = []
testy = []
test_size = 1000

for j in range(test_size):
    i = random.randint(0, len(test_data) - 1)
    X = test_data[i].reshape((-1, 1))
    y = test_labels[i]
    
    slice_start = random.randint(0, len(X) - slice_len)
    slice_end = slice_start + slice_len
    slice_x = X[slice_start:slice_end]
    slice_x = preprocess_signal(slice_x)
    slice_y = y
    
    x_250 = resample(slice_x, 250).reshape((1, -1, 1))
    x_500 = resample(slice_x, 500).reshape((1, -1, 1))
    x_1000 = slice_x.reshape((1, -1, 1))
    
    testX.append([x_250, x_500, x_1000])
    testy.append(slice_y)

Генераторы батчей для сети:

In [177]:
def generate_slice(slice_len, data, labels):
    i = random.randint(0, len(data) - 1)
    X = data[i].reshape((-1, 1))
    y = labels[i]
    
    slice_start = random.randint(0, len(X) - slice_len)
    slice_end = slice_start + slice_len
    slice_x = X[slice_start:slice_end]
    slice_x = preprocess_signal(slice_x)
    
    return slice_x, y

In [178]:
def generator(batch_size, slice_len, data, labels):
    while True:
        batch_x = []
        batch_y = []
        
        for i in range(0, batch_size):
            x, y = generate_slice(slice_len, data, labels)
            batch_x.append(x)
            batch_y.append(y)
            
        y = np.array(batch_y)
        
        x_250 = np.array([resample(i, 250) for i in batch_x])
        x_500 = np.array([resample(i, 500) for i in batch_x])
        x = np.array([i for i in batch_x])
        yield ([x_250, x_500, x], y)

In [179]:
from keras.layers import Convolution1D, Dense, Dropout, Input, merge, GlobalMaxPooling1D, MaxPooling1D, BatchNormalization
from keras.models import Model, load_model
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

Базовый блок сети, каждый из которых применяется на своём масштабе.

In [236]:
def get_base_model(input_len, fsize):
    input_seq = Input(shape=(input_len, 1))
    nb_filters = 50
    convolved = Convolution1D(nb_filters, fsize, border_mode="same", activation="tanh")(input_seq)
    processed = GlobalMaxPooling1D()(convolved)
    compressed = Dense(150, activation="tanh")(processed)
    compressed = Dropout(0.3)(compressed)
    compressed = Dense(150, activation="tanh")(compressed)
    model = Model(input=input_seq, output=compressed)            
    return model

In [237]:
input250_seq = Input(shape=(250, 1))
input500_seq = Input(shape=(500, 1))
input1000_seq = Input(shape=(1000, 1))
    
base_network250 = get_base_model(250, 15) # 4
base_network500 = get_base_model(500, 25) # 7
base_network1000 = get_base_model(1000, 35) # 10
embedding_250 = base_network250(input250_seq)
embedding_500 = base_network500(input500_seq)
embedding_1000 = base_network1000(input1000_seq)
    
merged = merge([embedding_250, embedding_500, embedding_1000], mode="concat")
merged = Dense(150, activation="tanh")(merged)
merged = Dropout(0.3)(merged)
out = Dense(1, activation='sigmoid')(merged)
model = Model(input=[input250_seq, input500_seq, input1000_seq], output=out)
    
# opt = RMSprop(lr=0.005, clipvalue=10**6)
opt = SGD(lr=0.001, momentum=0.9, nesterov=False)
# opt = Adam(lr=0.0001)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])

In [238]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_127 (InputLayer)           (None, 250, 1)        0                                            
____________________________________________________________________________________________________
input_128 (InputLayer)           (None, 500, 1)        0                                            
____________________________________________________________________________________________________
input_129 (InputLayer)           (None, 1000, 1)       0                                            
____________________________________________________________________________________________________
model_85 (Model)                 (None, 150)           31100       input_127[0][0]                  
___________________________________________________________________________________________

In [200]:
nb_epoch = 100
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
modelCheckpoing = ModelCheckpoint('weights.hdf5')
samples_per_epoch = 10000

model.fit_generator(generator(batch_size=100, slice_len=slice_len, data=train_data, labels=train_labels), 
                    samples_per_epoch, 
                    nb_epoch, 
                    validation_data=generator(batch_size=1000, slice_len=slice_len, data=test_data, labels=test_labels), 
                    nb_val_samples=10000,
                    callbacks=[earlyStopping, modelCheckpoing], 
                    verbose=1)

Epoch 1/100
10000/10000 [==============================] - 108s - loss: 0.6151 - acc: 0.6902 - val_loss: 0.6253 - val_acc: 0.6623

Тестирование на отложенной выборке:

In [244]:
model.load_weights('weights.hdf5')

In [245]:
pr = []
for j in range(len(testX)):
    pr.append(model.predict(testX[j])[0][0])
pr = np.array(pr)

In [246]:
roc_auc_score(testy, pr)

0.64245418229500195

In [247]:
space = np.linspace(np.min(pr), np.max(pr), 100)
max([accuracy_score(testy, pr > thr) for thr in space])

0.74299999999999999

Подготовим преобразователь датасета:

In [205]:
weights = model.get_weights()

In [206]:
def base_model_preprocess(input_len, nb_filters, fsize, weights):
    input_seq = Input(shape=(input_len, 1))
    convolved = Convolution1D(nb_filters, fsize, border_mode="same", activation="tanh", 
                              weights=[weights[0], weights[1]])(input_seq)
    processed = GlobalMaxPooling1D()(convolved)
    compressed = Dense(150, activation="tanh", weights=[weights[2], weights[3]])(processed)
    compressed = Dropout(0.3)(compressed)
    compressed = Dense(150, activation="tanh", weights=[weights[4], weights[5]])(compressed)
    model = Model(input=input_seq, output=compressed) 
    return model

In [207]:
len(weights)

22

In [208]:
input250_seq = Input(shape=(250, 1))
input500_seq = Input(shape=(500, 1))
input1000_seq = Input(shape=(1000, 1))

model_250 = base_model_preprocess(250, 50, 15, weights[0:6])
model_500 = base_model_preprocess(500, 50, 25, weights[6:12])
model_1000 = base_model_preprocess(1000, 50, 35, weights[12:18])

merged = merge([model_250(input250_seq), model_500(input500_seq), model_1000(input1000_seq)], mode="concat")
merged = Dense(150, activation="tanh", weights=[weights[18], weights[19]])(merged)

preprocess = Model(input=[input250_seq, input500_seq, input1000_seq], output=merged)

In [209]:
def preprocess_data(size, data, labels):
    prep = np.empty((size, 150))
    prep_y = np.empty(size)
    for j in tqdm.tqdm_notebook(xrange(size)):
        i = random.randint(0, len(data) - 1)
        X = data[i].reshape((-1, 1))
        y = labels[i]

        slice_start = random.randint(0, len(X) - slice_len)
        slice_end = slice_start + slice_len
        slice_x = X[slice_start:slice_end]
        slice_x = preprocess_signal(slice_x)
        
        x_250 = resample(slice_x, 250).reshape((1, -1, 1))
        x_500 = resample(slice_x, 500).reshape((1, -1, 1))
        x_1000 = slice_x.reshape((1, -1, 1))

        prep[j] = preprocess.predict([x_250, x_500, x_1000])
        prep_y[j] = y
        
    return prep, prep_y

In [210]:
X_train, y_train = preprocess_data(10000, train_data, train_labels)
X_test, y_test = preprocess_data(10000, test_data, test_labels)

In [211]:
clf = RandomForestClassifier(n_estimators=1000, criterion='entropy', n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [212]:
pr = clf.predict_proba(X_test)[:, 1]

In [213]:
roc_auc_score(y_test, pr)

0.57810347634704928

In [214]:
space = np.linspace(np.min(pr), np.max(pr), 100)
max([accuracy_score(y_test, pr > thr) for thr in space])

0.66569999999999996

То есть результат хуже, чем у сетки.

Была идея усреднять показания внутри одного сигнала:

In [215]:
window = 1000
step = 500

pr = []
for s in xrange(len(test_data)):
    count = int(float(len(test_data[s]) - window) / float(step))
    prob_sum = 0
    for j in xrange(count):
        slice_x = test_data[s][j*step:j*step+window].reshape((-1, 1))
        slice_x = np.array(slice_x, dtype=np.float32)
        slice_x = preprocess_signal(slice_x)
        
        x_250 = resample(slice_x, 250).reshape((1, -1, 1))
        x_500 = resample(slice_x, 500).reshape((1, -1, 1))
        x_1000 = slice_x.reshape((1, -1, 1))
        
        prob_sum += model.predict([x_250, x_500, x_1000])[0][0]
    pr.append(prob_sum / float(count))

print roc_auc_score(test_labels, pr)

0.71597392013


In [216]:
pr = np.array(pr)
space = np.linspace(np.min(pr), np.max(pr), 100)
max([accuracy_score(test_labels, pr > thr) for thr in space])

0.82714054927302105

Попробуем усреднить преобразованный сеткой датасет

In [276]:
window = 1000
step = 500
length = 150

In [277]:
def preprocess_avg(signal):
    count = int(float(len(signal) - window) / float(step))
    avg_signal = np.zeros((count, length))

    for j in xrange(count):
        slice_x = signal[j*step:j*step+window].reshape((-1, 1))
        slice_x = np.array(slice_x, dtype=np.float32)
        slice_x = preprocess_signal(slice_x)
        
        x_250 = resample(slice_x, 250).reshape((1, -1, 1))
        x_500 = resample(slice_x, 500).reshape((1, -1, 1))
        x_1000 = slice_x.reshape((1, -1, 1))
        
        avg_signal[j, :] += preprocess.predict([x_250, x_500, x_1000]).reshape(-1)
    
    return avg_signal.mean(axis=0)

In [278]:
X_tr = np.zeros((len(train_data), length))
X_ts = np.zeros((len(test_data), length))
y_tr = train_labels
y_ts = test_labels

for j in tqdm.tqdm_notebook(xrange(len(train_data))):
    X_tr[j, :] = preprocess_avg(train_data[j])
    
for j in tqdm.tqdm_notebook(xrange(len(test_data))):
    X_ts[j, :] = preprocess_avg(test_data[j])

In [279]:
clf = RandomForestClassifier(n_estimators=1000, criterion='entropy', n_jobs=-1)
clf.fit(X_tr, y_tr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [280]:
pr = clf.predict_proba(X_ts)[:, 1]

In [281]:
roc_auc_score(y_ts, pr)

0.5416404703690767

In [282]:
space = np.linspace(np.min(pr), np.max(pr), 100)
max([accuracy_score(y_ts, pr > thr) for thr in space])

0.69466882067851377

А теперь прикинем, что было бы, если бы работали с коэффициентами Фурье с таких же коротких окон.

In [356]:
random.seed(777)

train_size = 10000
signal_tmp = np.zeros((train_size, window))
trainy = np.zeros(train_size)

for j in tqdm.tqdm_notebook(xrange(train_size)):
    i = random.randint(0, len(train_data) - 1)
    slice_start = random.randint(0, len(train_data[i]) - slice_len)
    signal_tmp[j, :] = train_data[i][slice_start:slice_start+slice_len]
    trainy[j] = train_labels[i]
    
trainX = np.abs(fft(signal_tmp))[:, :500]

In [358]:
random.seed(777)

test_size = 10000
signal_tmp = np.zeros((test_size, window))
testy = np.zeros(test_size)

for j in tqdm.tqdm_notebook(xrange(test_size)):
    i = random.randint(0, len(test_data) - 1)
    slice_start = random.randint(0, len(test_data[i]) - slice_len)
    signal_tmp[j, :] = test_data[i][slice_start:slice_start+slice_len]
    testy[j] = test_labels[i]
    
testX = np.abs(fft(signal_tmp))[:, :500]

In [359]:
clf = RandomForestClassifier(n_estimators=1000, criterion='entropy', n_jobs=-1)
clf.fit(trainX, trainy)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [360]:
pr = clf.predict_proba(testX)[:, 1]

In [361]:
roc_auc_score(testy, pr)

0.81123145586239587

In [362]:
space = np.linspace(np.min(pr), np.max(pr), 100)
max([accuracy_score(y_test, pr > thr) for thr in space])

0.66010000000000002

Другое дело было, когда эти признки усреднялись для разных окон.

А теперь усредним предсказания по таким же окнам.

In [363]:
pr = []
for s in tqdm.tqdm_notebook(xrange(len(test_data))):    
    sig = test_data[s]
    count_part_1 = sig.shape[0] / window
    count_part_2 = (sig.shape[0] - step) / window
    rsig = np.vstack((sig[:count_part_1*window].reshape((count_part_1, window)),
                      sig[step:step+count_part_2*window].reshape((count_part_2, window))))    
    pr.append(clf.predict_proba(np.abs(fft(rsig, axis=1))[:, :500])[:, 1].mean()) 

In [364]:
roc_auc_score(test_labels, pr)

0.84392828035859813

In [365]:
space = np.linspace(np.min(pr), np.max(pr), 100)
max([accuracy_score(test_labels, pr > thr) for thr in space])

0.79806138933764137